In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import os
import json

### Connect to Kaggle account and get the dataset

In [4]:
kaggle datasets download -d brijlaldhankour/flood-prediction-factors


  0%|          | 0.00/680k [00:00<?, ?B/s]
100%|██████████| 680k/680k [00:01<00:00, 528kB/s]
100%|██████████| 680k/680k [00:01<00:00, 527kB/s]
